# Voice Recognizer System

In [11]:
!pip install tk
!pip install -U openai-whisper
!pip install ffmpeg
!pip install pyannote-audio
!pip install torch
!pip install pydub

In [1]:
from tkinter import *
from tkinter import messagebox
from tkinter import filedialog
from tkinter import ttk
import subprocess
import processing_file
import random
import os
from pyannote.audio import Pipeline
from pydub import AudioSegment
from pyannote.audio import Model
import whisper
import numpy as np
import gc
from pyannote.audio import Inference
from numpy.linalg import norm
from pyannote.core import Segment
import warnings
warnings.filterwarnings('ignore')
from math import ceil

D:\anaconda\Lib\site-packages\pyannote\audio\core\io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [2]:
#-------------global variables
creds={}
user_name=[]

def ret_login_dets():
    #if folder locked
    if not os.path.exists("Private"):
        #unlock Folder
        cmd1 = ['FolderLock.bat']
        p = subprocess.Popen(cmd1,stdin=subprocess.PIPE,stdout=subprocess.PIPE)
        output = p.communicate(input='virkhase6a'.encode())[0]

    #retrieve login details
    with open('Private\login_details.txt', 'r') as f:
        lines=f.readlines()
        for line in lines:
            u=line.split('\t')[0]
            p=line.split('\t')[1]
            p=p.strip()
            creds[u]=p
        
    #lock Folder
    cmd1 = ['FolderLock.bat']
    p = subprocess.Popen(cmd1,stdin=subprocess.PIPE,stdout=subprocess.PIPE)
    output = p.communicate(input='y'.encode())[0]
    return True #to ensure that no work occurs until data required retrieved

def open_file(drop, filepath, rof_canvas):
    fn=drop.get()
    #if folder locked
    if not os.path.exists("Private"):
        #unlock Folder
        cmd1 = ['FolderLock.bat']
        p = subprocess.Popen(cmd1,stdin=subprocess.PIPE,stdout=subprocess.PIPE)
        output = p.communicate(input='virkhase6a'.encode())[0]
    #open file
    file_open_path=filepath+"\\"+fn
    lines=''
    with open(file_open_path, 'r') as f:
        lines=f.readlines()
        f.close()
    #lock Folder
    cmd1 = ['FolderLock.bat']
    p = subprocess.Popen(cmd1,stdin=subprocess.PIPE,stdout=subprocess.PIPE)
    output = p.communicate(input='y'.encode())[0]
    #output
    with open('temp_file.txt', 'w') as f2:
        for line in lines:
            f2.write(line)
        f2.close()
    #show possible files
    title=Label(rof_canvas, text='Saved in Current Folder as: temp_file.txt',font='times 16 bold italic', bg='#210021', fg='#a5c9c3',highlightthickness=0)
    title.place(relx=0.5, rely=0.8, anchor=CENTER)
    
def ret_old_files():
    #back to options button
    backop_canvas=Canvas(window,width=15,height=1,background='#9c2f2f', bd=0, highlightthickness=0)
    backop_button=Button(backop_canvas,text='Back to Options',command=lambda: back_op(backop_canvas),background='#9c2f2f',fg='white', activebackground='#bf1d1d', 
                    activeforeground='white',highlightthickness=2,width=15,height=1,border=0,cursor='hand2',font=('Comic Sans MS',13,'bold italic'))
    backop_button.bind("<Enter>",func=lambda e:backop_button.config(background='#bf1d1d'))
    backop_button.bind("<Leave>",func=lambda e:backop_button.config(background='#9c2f2f'))
    backop_button.pack()
    backop_canvas.place(relx=0.1, rely=0.06, anchor=CENTER)
    #ret old files canvas
    rof_canvas=Canvas(window,width=500,height=400,background='#210021', bd=0, highlightthickness=0)
    rof_canvas.place(relx=0.5, rely=0.54, anchor=CENTER)
    #ret files title
    ret_files_title=Label(rof_canvas, text='Retrieve Past Saved Outputs',font='times 20 bold italic', 
                          bg='#44274f', fg='#e9d5f0',highlightthickness=1, padx=70)
    ret_files_title.place(relx=0.5, rely=0.06, anchor=CENTER)
    
    #if folder locked
    if not os.path.exists("Private"):
        #unlock Folder
        cmd1 = ['FolderLock.bat']
        p = subprocess.Popen(cmd1,stdin=subprocess.PIPE,stdout=subprocess.PIPE)
        output = p.communicate(input='virkhase6a'.encode())[0]
    #open username's folder
    fp="Private\\user_data\\"+user_name[0]+"_user"
    files_av=os.listdir(fp)
    #lock Folder
    cmd1 = ['FolderLock.bat']
    p = subprocess.Popen(cmd1,stdin=subprocess.PIPE,stdout=subprocess.PIPE)
    output = p.communicate(input='y'.encode())[0]
    
    #show possible files
    title=Label(rof_canvas, text='Available Files',font='times 21 bold italic', bg='#210021', fg='#d6c7d5',highlightthickness=0)
    title.place(relx=0.5, rely=0.3, anchor=CENTER)
    #select menu
    drop=ttk.Combobox(rof_canvas,values=files_av, state='readonly',height=50,width=25,font=('times',15))
    drop.set("Choose File to open")
    drop.place(relx=0.5, rely=0.42, anchor=CENTER)
    #open file button
    openf_button=Button(rof_canvas,text='Retrieve File..',command=lambda: open_file(drop, fp, rof_canvas),background='#44274f',fg='white', activebackground='#704980', 
                        activeforeground='white',highlightthickness=2,width=13,height=1,border=0,cursor='hand2',font=('times',16))
    openf_button.bind("<Enter>",func=lambda e:openf_button.config(background='#704980'))
    openf_button.bind("<Leave>",func=lambda e:openf_button.config(background='#44274f'))
    openf_button.place(relx=0.5, rely=0.6, anchor=CENTER)

def save_output(out_file, process_canvas):
    #if folder locked
    if not os.path.exists("Private"):
        #unlock Folder
        cmd1 = ['FolderLock.bat']
        p = subprocess.Popen(cmd1,stdin=subprocess.PIPE,stdout=subprocess.PIPE)
        output = p.communicate(input='virkhase6a'.encode())[0]
    #take input from file
    lines=''
    with open(out_file, 'r') as f:
        lines=f.readlines()
        f.close()
    #check for unique filename
    while True:
        filename=user_name[0]+str(random.randint(0,15000))
        fp="Private\\user_data\\"+user_name[0]+"_user\\"+filename
        if not os.path.exists(fp):
            break
    #write data into file
    with open(fp, 'w') as f:
        for line in lines:
            f.write(line)
        f.close()
    #lock Folder
    cmd1 = ['FolderLock.bat']
    p = subprocess.Popen(cmd1,stdin=subprocess.PIPE,stdout=subprocess.PIPE)
    output = p.communicate(input='y'.encode())[0]
    #confirmation label
    fn=fp.split("\\")[-1]
    save_label=Label(process_canvas, text="SAVED!",font='times 21 bold italic', bg='#210021', fg='#a5c9c3',highlightthickness=0)
    save_label.place(relx=0.5, rely=0.7, anchor=CENTER)
    fn_label=Label(process_canvas, text="filename: "+fn+"\n and tr_file.txt on your current folder",
                   font='times 16 bold italic', bg='#210021', fg='#a5c9c3',highlightthickness=0)
    fn_label.place(relx=0.5, rely=0.8, anchor=CENTER)

#-----------------------------------AUDIO TO TEXT + SPEAKER DIARIZATION------------------------    
def file_process(filepath):
    res=processing_file.do_work(filepath)
    #canvas
    process_canvas=Canvas(window,width=500,height=400,background='#210021', bd=0, highlightthickness=0)
    process_canvas.place(relx=0.5, rely=0.54, anchor=CENTER)
    if res==0:
        #confirmation label
        done_label=Label(process_canvas, text="DONE PROCESSING!",font='times 21 bold italic', bg='#210021', fg='#a5c9c3',highlightthickness=0)
        done_label.place(relx=0.5, rely=0.3, anchor=CENTER)
        #show output button
        show_button=Button(process_canvas,text='Save Output..',command=lambda: save_output('tr_file.txt',process_canvas),background='#450645',fg='white',activebackground='#540854', 
                            activeforeground='white',highlightthickness=2,width=13,height=1,border=0,cursor='hand2',font=('times',18))
        show_button.bind("<Enter>",func=lambda e:show_button.config(background='#540854'))
        show_button.bind("<Leave>",func=lambda e:show_button.config(background='#450645'))
        show_button.place(relx=0.5, rely=0.5, anchor=CENTER)
    elif res==-1:
        #error label
        e_label=Label(process_canvas, text="ERROR!\nTRY AGAIN!",font='times 21 bold italic', bg='#210021', fg='#a5c9c3',highlightthickness=0)
        e_label.place(relx=0.5, rely=0.45, anchor=CENTER)

#-----------------------------------------END---------------------------------------------------

def browse_files(label_file_explorer, fp_canvas):
    valid=False
    while not valid:
        filepath = filedialog.askopenfilename(initialdir = "/",
                                              title = "Select a File",
                                              filetypes=(("Audio Files","*.mp3*"),("All Files","*.*")))
        filename=filepath.split('/')[-1]
        label_file_explorer.configure(text="File Selected: \n"+filename)
        if os.path.getsize(filepath)!=0 and filepath[-3:]=='mp3' and filepath!='': #wav file, not empty, not over 25mb, none selected
            valid=True
        else:
            error_label=Label(fp_canvas, text="Invalid File selected",font='times 16 bold italic', bg='#210021', fg='red',highlightthickness=0)
            error_label.place(relx=0.5, rely=0.7, anchor=CENTER)

    #process button
    bf_button=Button(fp_canvas,text='Start Processing..',command=lambda: file_process(filepath), background='#450645',fg='white',
                     activebackground='#540854', activeforeground='white',highlightthickness=2,width=13,height=1,border=0,cursor='hand2',
                     font=('times',18))
    bf_button.bind("<Enter>",func=lambda e:bf_button.config(background='#540854'))
    bf_button.bind("<Leave>",func=lambda e:bf_button.config(background='#450645'))
    bf_button.place(relx=0.5, rely=0.7, anchor=CENTER)

def new_file_process():
    #back to options button
    backop_canvas=Canvas(window,width=15,height=1,background='#9c2f2f', bd=0, highlightthickness=0)
    backop_button=Button(backop_canvas,text='Back to Options',command=lambda: back_op(backop_canvas),background='#9c2f2f',fg='white', activebackground='#bf1d1d', 
                    activeforeground='white',highlightthickness=2,width=15,height=1,border=0,cursor='hand2',font=('Comic Sans MS',13,'bold italic'))
    backop_button.bind("<Enter>",func=lambda e:backop_button.config(background='#bf1d1d'))
    backop_button.bind("<Leave>",func=lambda e:backop_button.config(background='#9c2f2f'))
    backop_button.pack()
    backop_canvas.place(relx=0.1, rely=0.06, anchor=CENTER)
    #fileprocess canvas
    fp_canvas=Canvas(window,width=500,height=400,background='#210021', bd=0, highlightthickness=0)
    fp_canvas.place(relx=0.5, rely=0.54, anchor=CENTER)
    #title
    wel_title=Label(fp_canvas, text="Select File to Process",font='times 20 bold italic', bg='#44274f', fg='#e9d5f0',highlightthickness=1, padx=100)
    wel_title.place(relx=0.5, rely=0.06, anchor=CENTER)
    #file explorer label
    label_file_explorer=Label(fp_canvas, text='', font='times 14', bg='#210021',  fg='#f0e4f5')
    label_file_explorer.place(relx=0.5, rely=0.55, anchor=CENTER)
    #browse files button
    bf_button=Button(fp_canvas,text='Browse Files',command=lambda: browse_files(label_file_explorer, fp_canvas),background='#450645',fg='white',activebackground='#540854', 
                        activeforeground='white',highlightthickness=2,width=13,height=1,border=0,cursor='hand2',font=('times',18))
    bf_button.bind("<Enter>",func=lambda e:bf_button.config(background='#540854'))
    bf_button.bind("<Leave>",func=lambda e:bf_button.config(background='#450645'))
    bf_button.place(relx=0.5, rely=0.35, anchor=CENTER)

def back_op(backop_canvas):
    backop_canvas.destroy()
    main_menu()

def main_menu():
    main_canvas=Canvas(window,width=500,height=400,background='#210021', bd=0, highlightthickness=0)
    main_canvas.place(relx=0.5, rely=0.54, anchor=CENTER)
    #welcome box
    welcome_text="Welcome "+user_name[0].capitalize()
    wel_title=Label(main_canvas, text=welcome_text,font='times 20 bold italic', bg='#44274f', fg='#e9d5f0',highlightthickness=1, padx=140)
    wel_title.place(relx=0.5, rely=0.06, anchor=CENTER)
    #option title
    uname=Label(main_canvas, text='Options:', font='times 18 bold', bg='#210021',  fg='#f0e4f5')
    uname.place(relx=0.5, rely=0.32, anchor=CENTER)
    #options-retrieve old files button
    retrieve_button=Button(main_canvas,text='Retrieve Old Files',command=ret_old_files,background='#210021',fg='white',activebackground='#540854', 
                        activeforeground='white',highlightthickness=2,width=13,height=1,border=0,cursor='hand2',font=('times',18))
    retrieve_button.bind("<Enter>",func=lambda e:retrieve_button.config(background='#540854'))
    retrieve_button.bind("<Leave>",func=lambda e:retrieve_button.config(background='#210021'))
    retrieve_button.place(relx=0.5, rely=0.45, anchor=CENTER)
    #options-new file processing
    nfp_button=Button(main_canvas,text='Process New File',command=new_file_process,background='#210021',fg='white',activebackground='#540854', 
                        activeforeground='white',highlightthickness=2,width=13,height=1,border=0,cursor='hand2',font=('times',18))
    nfp_button.bind("<Enter>",func=lambda e:nfp_button.config(background='#540854'))
    nfp_button.bind("<Leave>",func=lambda e:nfp_button.config(background='#210021'))
    nfp_button.place(relx=0.5, rely=0.57, anchor=CENTER)

def login_valid(username_entry, pass_entry, login_canvas):
    un=username_entry.get()
    pwd=pass_entry.get()
    if un=='' or pwd=='':
        messagebox.showinfo("Empty required fields", "Please enter your username/password again to proceed further")
        username_entry.delete(0, END)
        pass_entry.delete(0, END)
    elif un not in creds.keys() or creds[un]!=pwd:
        messagebox.showinfo("Login Error", "Please check your username/password again")
        username_entry.delete(0, END)
        pass_entry.delete(0, END)
    elif creds[un]==pwd:
        user_name.append(un)
        login_canvas.destroy()
        main_menu()

def login():
    #login screen
    login_canvas=Canvas(window,width=900,height=500,background='#210021', bd=0, highlightthickness=0)
    login_canvas.place(relx=0.5, rely=0.54, anchor=CENTER)
    login_title=Label(login_canvas, text='Login Form',font='times 20 bold italic', bg='#44274f', fg='#e9d5f0',highlightthickness=1, padx=360)
    login_title.place(relx=0.5, rely=0.04, anchor=CENTER)
    uname=Label(login_canvas, text='User Name', font='times 18 bold', bg='#210021',  fg='#f0e4f5')
    uname.place(relx=0.2, rely=0.4, anchor=CENTER)
    username_entry=Entry(login_canvas, justify = CENTER, font = ('courier', 16, 'italic'))
    username_entry.place(relx=0.6, rely=0.4, anchor=CENTER)
    password=Label(login_canvas, text='Password', font='times 18 bold', bg='#210021',  fg='#f0e4f5')
    password.place(relx=0.2, rely=0.5, anchor=CENTER)
    pass_entry=Entry(login_canvas, justify = CENTER, font = ('courier', 16, 'italic'), show='*')
    pass_entry.place(relx=0.6, rely=0.5, anchor=CENTER)
    login_button=Button(login_canvas,text='Login',command=lambda: login_valid(username_entry, pass_entry, login_canvas),background='#44274f',fg='white', activebackground='#704980', 
                        activeforeground='white',highlightthickness=2,width=13,height=1,border=0,cursor='hand2',font=('times',16))
    login_button.bind("<Enter>",func=lambda e:login_button.config(background='#704980'))
    login_button.bind("<Leave>",func=lambda e:login_button.config(background='#44274f'))
    login_button.place(relx=0.5, rely=0.8, anchor=CENTER)

def shut_down_app():
    window.destroy()

#----------------------------------------MAIN DRIVER FUNCTION--------------------------------------    
if __name__=="__main__":
    #retrieve login details
    ret_login_dets()
    
    #tkinter window
    window=Tk()
    window.geometry('1150x700')
    window.title('Voice Recognizer System')

    # background image
    bgimage=PhotoImage(file='images/wallpaper.ppm')
    bimg=Label(window, image=bgimage)
    bimg.place(relheight=1, relwidth=1)
    
    #title
    label=Label(window,text='Voice Recognizer System',font='times 30 bold italic',fg='#d8e2eb', bg='#1a001a')
    label.place(relx=0.5, rely=0.06, anchor=CENTER)

    #poweroff button
    poweroff_button=Button(window,text='Poweroff',command=shut_down_app,background='#9c2f2f',fg='white', activebackground='#bf1d1d', 
                        activeforeground='white',highlightthickness=2,width=10,height=1,border=0,cursor='hand2',font=('Comic Sans MS',13,'bold italic'))
    poweroff_button.bind("<Enter>",func=lambda e:poweroff_button.config(background='#bf1d1d'))
    poweroff_button.bind("<Leave>",func=lambda e:poweroff_button.config(background='#9c2f2f'))
    poweroff_button.place(relx=0.935, rely=0.06, anchor=CENTER)
    
    login()

    window.mainloop()

torchvision is not available - cannot save figures


['[ 00:00:00.110 -->  00:00:00.534] A SPEAKER_01', '[ 00:00:01.162 -->  00:00:14.134] B SPEAKER_01', '[ 00:00:14.558 -->  00:00:30.568] C SPEAKER_01', '[ 00:00:30.636 -->  00:00:32.792] D SPEAKER_01', '[ 00:00:33.200 -->  00:00:35.662] E SPEAKER_01', '[ 00:00:35.662 -->  00:00:35.696] F SPEAKER_00', '[ 00:00:36.324 -->  00:00:38.022] G SPEAKER_00']


Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v2.2.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\IT\.cache\torch\pyannote\models--pyannote--embedding\snapshots\4db4899737a38b2d618bbd74350915aa10293cb2\pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.2.2+cpu. Bad things might happen unless you revert torch to 1.x.


Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v2.2.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\IT\.cache\torch\pyannote\models--pyannote--embedding\snapshots\4db4899737a38b2d618bbd74350915aa10293cb2\pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.2.2+cpu. Bad things might happen unless you revert torch to 1.x.
SPEAKER_00 's Segment:  00:00:35.662-->00:00:35.696  Not Used for Speaker Recognition
SPEAKER_01 =>
	 Bill Gates :  0.39159349767354956
	 JK-bts :  0.05388290059719851
	 Mark Zuckerberg :  0.3127649069534899
	 Namjoon-bts :  0.03687230151281304
	 Selena Gomez :  0.03318856959073125
	 Taylor Swift :  0.17522311824303147
SPEAKER_01 : Bill Gates
SPEAKER_00 =>
	 Bill Gates :  0.2633027
	 JK-bts :  0.06070598
	 Mark Zuckerberg :  0.25557694
	 Namjoon-bts :  -0.051270276
	 Selena Gomez :  0.06577813
	 Taylor Swift :  -0.025513899
SPEAKER_00 : Mark Zuckerberg
Doing work on:  1 / 7
Doing work on:  2 / 7
Doing work on:  3 / 7
Doing work on:  4 / 7
Doing work on:  5 / 7
Doing work on:  6 / 7
Doing work on:  7 / 7
Final Identified Speakers: 
 {'SPEAKER_01':